In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("helloworld").getOrCreate()

In [2]:
df = spark.read.csv('C:/Users/ACER/Desktop/kaggle/transactions_data.csv',header=True)

In [3]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- client_id: string (nullable = true)
 |-- card_id: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- use_chip: string (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- merchant_city: string (nullable = true)
 |-- merchant_state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- mcc: string (nullable = true)
 |-- errors: string (nullable = true)



In [4]:
df.show(5)

+-------+-------------------+---------+-------+-------+-----------------+-----------+-------------+--------------+-------+----+------+
|     id|               date|client_id|card_id| amount|         use_chip|merchant_id|merchant_city|merchant_state|    zip| mcc|errors|
+-------+-------------------+---------+-------+-------+-----------------+-----------+-------------+--------------+-------+----+------+
|7475327|2010-01-01 00:01:00|     1556|   2972|$-77.00|Swipe Transaction|      59935|       Beulah|            ND|58523.0|5499|  NULL|
|7475328|2010-01-01 00:02:00|      561|   4575| $14.57|Swipe Transaction|      67570|   Bettendorf|            IA|52722.0|5311|  NULL|
|7475329|2010-01-01 00:02:00|     1129|    102| $80.00|Swipe Transaction|      27092|        Vista|            CA|92084.0|4829|  NULL|
|7475331|2010-01-01 00:05:00|      430|   2860|$200.00|Swipe Transaction|      27092|  Crown Point|            IN|46307.0|4829|  NULL|
|7475332|2010-01-01 00:06:00|      848|   3915| $46.41|

In [5]:
from pyspark.sql import SparkSession
from IPython.display import display

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("ExampleApp") \
    .getOrCreate()

In [6]:
df2= spark.read.csv('C:/Users/ACER/Desktop/kaggle/cards_data.csv', header=True, inferSchema=True)
df2.createOrReplaceTempView("cards")
df2.show(5)

+----+---------+----------+---------------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+
|  id|client_id|card_brand|      card_type|     card_number|expires|cvv|has_chip|num_cards_issued|credit_limit|acct_open_date|year_pin_last_changed|card_on_dark_web|
+----+---------+----------+---------------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+
|4524|      825|      Visa|          Debit|4344676511950444|12/2022|623|     YES|               2|      $24295|       09/2002|                 2008|              No|
|2731|      825|      Visa|          Debit|4956965974959986|12/2020|393|     YES|               2|      $21968|       04/2014|                 2014|              No|
|3701|      825|      Visa|          Debit|4582313478255491|02/2024|719|     YES|               2|      $46414|       07/2003|                 2004|              No|
|  4

In [7]:
display(spark.sql("""
SELECT
    id,
    client_id,
    card_brand,
    TRIM(REGEXP_REPLACE(card_type, '\\(Prepaid\\)', '')) AS card_type,
    CASE
        WHEN card_type LIKE '%Prepaid%' THEN 'yes'
        ELSE 'no'
    END AS isPrepaid,
    card_number AS card_number_masked,
    expires,
    cvv,
    has_chip,
    num_cards_issued,
    CAST(REGEXP_REPLACE(credit_limit, '[$,]', '') AS DECIMAL(10, 2)) AS credit_limit,
    TO_DATE(acct_open_date, 'yyyy-MM-dd') AS acct_open_date,
    year_pin_last_changed,
    card_on_dark_web
FROM cards
LIMIT(10)
""").toPandas())

,id,client_id,card_brand,card_type,isPrepaid,card_number_masked,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,no,4344676511950444,12/2022,623,YES,2,24295.00,None,2008,No
1,2731,825,Visa,Debit,no,4956965974959986,12/2020,393,YES,2,21968.00,None,2014,No
2,3701,825,Visa,Debit,no,4582313478255491,02/2024,719,YES,2,46414.00,None,2004,No
3,42,825,Visa,Credit,no,4879494103069057,08/2024,693,NO,1,12400.00,None,2012,No
4,4659,825,Mastercard,Debit (),yes,5722874738736011,03/2009,75,YES,1,28.00,None,2009,No
5,4537,1746,Visa,Credit,no,4404898874682993,09/2003,736,YES,1,27500.00,None,2012,No
6,1278,1746,Visa,Debit,no,4001482973848631,07/2022,972,YES,2,28508.00,None,2011,No
7,3687,1746,Mastercard,Debit,no,5627220683410948,06/2022,48,YES,2,9022.00,None,2015,No
8,3465,1746,Mastercard,Debit (),yes,5711382187309326,11/2020,722,YES,2,54.00,None,2015,No
9,3754,1746,Mastercard,Debit (),yes,5766121508358701,02/2023,908,YES,1,99.00,None,2012,No


In [8]:
# CARDS DATA
display(spark.sql("""
SELECT
    id,
    client_id,
    card_brand,
    TRIM(REGEXP_REPLACE(card_type, '\\(Prepaid\\)', '')) AS card_type,
    CASE
        WHEN card_type LIKE '%(Prepaid)' THEN 'Yes'
        ELSE 'No'
    END AS is_prepaid,
    CAST(REPLACE(credit_limit, '$', '') AS DECIMAL(10, 2)) AS credit_limit_numeric
FROM cards
LIMIT 5;
""").toPandas())

,id,client_id,card_brand,card_type,is_prepaid,credit_limit_numeric
0,4524,825,Visa,Debit,No,24295.00
1,2731,825,Visa,Debit,No,21968.00
2,3701,825,Visa,Debit,No,46414.00
3,42,825,Visa,Credit,No,12400.00
4,4659,825,Mastercard,Debit (),Yes,28.00


In [9]:
#USER DATA
df3= spark.read.csv('C:/Users/ACER/Desktop/kaggle/users_data.csv', header=True, inferSchema=True)
df3.createOrReplaceTempView("users")
df3.show(5)


+----+-----------+--------------+----------+-----------+------+--------------------+--------+---------+-----------------+-------------+----------+------------+----------------+
|  id|current_age|retirement_age|birth_year|birth_month|gender|             address|latitude|longitude|per_capita_income|yearly_income|total_debt|credit_score|num_credit_cards|
+----+-----------+--------------+----------+-----------+------+--------------------+--------+---------+-----------------+-------------+----------+------------+----------------+
| 825|         53|            66|      1966|         11|Female|       462 Rose Lane|   34.15|  -117.76|           $29278|       $59696|   $127613|         787|               5|
|1746|         53|            68|      1966|         12|Female|3606 Federal Boul...|   40.76|   -73.74|           $37891|       $77254|   $191349|         701|               5|
|1718|         81|            67|      1938|         11|Female|     766 Third Drive|   34.02|  -117.89|           $

In [10]:
display(spark.sql("""
SELECT
    id,
    gender,
    CONCAT(birth_year, ' AD') AS birth_year,
    REGEXP_REPLACE(address, '[^A-Za-z]', '') AS Address,
    TRIM(REGEXP_REPLACE(REGEXP_REPLACE(address, '[^0-9]+', ' '), '[ ]+', ' ')) AS postal_code,
    CAST(REPLACE(yearly_income, '$', '') AS DECIMAL(10, 2)) AS yearly_income,
    CAST(REPLACE(total_debt, '$', '') AS DECIMAL(10, 2)) AS total_debt,
    credit_score,
    num_credit_cards,
    retirement_age,
    TRIM(REGEXP_REPLACE(address, '[^A-Za-z]', '')) AS address_letters_only
FROM users
LIMIT 5
""").toPandas())


,id,gender,birth_year,Address,postal_code,yearly_income,total_debt,credit_score,num_credit_cards,retirement_age,address_letters_only
0,825,Female,1966 AD,RoseLane,462,59696.00,127613.00,787,5,66,RoseLane
1,1746,Female,1966 AD,FederalBoulevard,3606,77254.00,191349.00,701,5,68,FederalBoulevard
2,1718,Female,1938 AD,ThirdDrive,766,33483.00,196.00,698,5,67,ThirdDrive
3,708,Female,1957 AD,MadisonStreet,3,249925.00,202328.00,722,4,63,MadisonStreet
4,1164,Male,1976 AD,ValleyStreamDrive,9620,109687.00,183855.00,675,1,70,ValleyStreamDrive


In [11]:
df4= spark.read.csv('C:/Users/ACER/Desktop/kaggle/transactions_data.csv', header=True, inferSchema=True)
df4.createOrReplaceTempView("transactions")
df4.show(5)

+-------+-------------------+---------+-------+-------+-----------------+-----------+-------------+--------------+-------+----+------+
|     id|               date|client_id|card_id| amount|         use_chip|merchant_id|merchant_city|merchant_state|    zip| mcc|errors|
+-------+-------------------+---------+-------+-------+-----------------+-----------+-------------+--------------+-------+----+------+
|7475327|2010-01-01 00:01:00|     1556|   2972|$-77.00|Swipe Transaction|      59935|       Beulah|            ND|58523.0|5499|  NULL|
|7475328|2010-01-01 00:02:00|      561|   4575| $14.57|Swipe Transaction|      67570|   Bettendorf|            IA|52722.0|5311|  NULL|
|7475329|2010-01-01 00:02:00|     1129|    102| $80.00|Swipe Transaction|      27092|        Vista|            CA|92084.0|4829|  NULL|
|7475331|2010-01-01 00:05:00|      430|   2860|$200.00|Swipe Transaction|      27092|  Crown Point|            IN|46307.0|4829|  NULL|
|7475332|2010-01-01 00:06:00|      848|   3915| $46.41|

In [12]:
display(spark.sql("""
SELECT
                  id,
                  date,
                  client_id,
                  card_id,
                  merchant_id,
                  amount,
                  use_chip
                  FROM transactions
                  LIMIT(5)
                  
""").toPandas())

,id,date,client_id,card_id,merchant_id,amount,use_chip
0,7475327,2010-01-01 00:01:00,1556,2972,59935,$-77.00,Swipe Transaction
1,7475328,2010-01-01 00:02:00,561,4575,67570,$14.57,Swipe Transaction
2,7475329,2010-01-01 00:02:00,1129,102,27092,$80.00,Swipe Transaction
3,7475331,2010-01-01 00:05:00,430,2860,27092,$200.00,Swipe Transaction
4,7475332,2010-01-01 00:06:00,848,3915,13051,$46.41,Swipe Transaction
